In [16]:
import os
import re

In [34]:
# # THIS IS UNNECESSARY IN CURRENT CONFIG (which can handle updates of any type)

# input_types = {
#     # ---------------------------------------------------------------------
#     # --- TEMPLATE.inc input types (dictates how values are added to line)
#     # 
#     # --- lake-specific parameters
#     "oblq": 'numeric',    # obliquity
#     "xlat": 'numeric',    # latitude (negative for south)
#     "xlon": 'numeric',    # longitude (negative for west)
#     "gmt": 'string',     # [hrs] local time relative to gmt in hours
#     "max_dep": 'numeric',  # [m] depth of lake at sill 
#     "basedep": 'numeric', # [m] elevation of basin at bottom 
#     "b_area": 'numeric', # [ha] area of catchment + lake
#     "cdrn": 'numeric',  # neutral drag coefficient (1.8 HAD 1.7GISS 1.2CCSM)
#     "eta": 'numeric',    # [m-1] shortwave extinction coefficient
#     "f": 'numeric',        # fraction of advected air over lake
#     "alb_slush": 'numeric', # [] albedo of melting snow
#     "alb_snow": 'numeric', # [] albedo of non-melting snow
#     "depth_begin": 'numeric', # [m] prescribed depth
#     "salty_begin": 'numeric', # [ppt] prescribed salinity
#     "o18air": 'numeric', # [per mil VSMOW] d18O of air above lake
#     "deutair": 'numeric', # [per mil VSMOW] dD of air above lake
#     "tempinit": 'numeric', # [degC] temperature to initialize lake at in INIT_LAKE subroutine
#     "deutinit": 'numeric', # [per mil VSMOW] dD to initialize lake at in INIT_LAKE subroutine
#     "o18init": 'numeric', # [per mil VSMOW] d18O to initialize lake at in INIT_LAKE subroutine
#     # 
#     # --- simulation-specific parameters
#     "nspin": 'numeric',     # [yrs] number of years for spinup
#     "bndry_flag": 'bool', # [".true." | ".false."] true for explicit boundary layer computation (presently only for sigma coord climate models)
#     "sigma": 'numeric', # sigma level for boundary flag
#     "wb_flag": 'bool', # [".true." | ".false."] true for variable lake depth
#     "iceflag": 'bool', # [".true." | ".false."] true for variable ice cover
#     "s_flag": 'bool', # [".true." | ".false."] true for variable salinity
#     "o18flag": 'bool', # [".true." | ".false."] true for variable d18O
#     "deutflag": 'bool', # [".true." | ".false."] true for variable dD
#     "z_screen": 'numeric',  # [m] height of meteorological inputs

# }

In [35]:
input_values = {
    # ---------------------------------------------------------------------
    # --- general model setup
    # 
    'outdir': '/Users/tylerkukla/Documents/GitHub/PRYSM/psm/lake',  # location to save output data
    'casename': '_test_case',  # name of the run case
    'lake_inc_filename': "lake_setup.inc",   # name of the '.inc' file to generate
    'lake_inc_template': "template.inc",     # name of the '.inc' template to modify
    'datafile': "CP_SLIM_lowTopo_500ppm_test_input.txt",  # name of the climate input data file (make sure single quotes are w/in double and \n is at end of single quote)
    'output_prof': "_test_prof.dat",  # name of the output profile file
    'output_surf': "_test_surf.dat",  # name of the output surface file
    'special_format_1': ['datafile', 'output_prof', 'output_surf'],  # keys that require single quotes w/in double and \n at end of single quote

    # ---------------------------------------------------------------------
    # --- TEMPLATE.inc inputs
    # 
    # --- lake-specific parameters
    "oblq": 23.4,    # obliquity
    "xlat": -6.30,   # latitude (negative for south)
    "xlon": 29.5,    # longitude (negative for west)
    "gmt": "+3",     # [hrs] local time relative to gmt in hours
    "max_dep": 999,  # [m] depth of lake at sill 
    "basedep": 733., # [m] elevation of basin at bottom 
    "b_area": 23100000, # [ha] area of catchment + lake
    "cdrn": 2.0e-3,  # neutral drag coefficient (1.8 HAD 1.7GISS 1.2CCSM)
    "eta": 0.065,    # [m-1] shortwave extinction coefficient
    "f": 0.3,        # fraction of advected air over lake
    "alb_slush": 0.4, # [] albedo of melting snow
    "alb_snow": 0.7, # [] albedo of non-melting snow
    "depth_begin": 570, # [m] prescribed depth
    "salty_begin": 0.0, # [ppt] prescribed salinity
    "o18air": -14.0, # [per mil VSMOW] d18O of air above lake
    "deutair": -96., # [per mil VSMOW] dD of air above lake
    "tempinit": 18., # [degC] temperature to initialize lake at in INIT_LAKE subroutine
    "deutinit": -100., # [per mil VSMOW] dD to initialize lake at in INIT_LAKE subroutine
    "o18init": -10., # [per mil VSMOW] d18O to initialize lake at in INIT_LAKE subroutine
    # 
    # --- simulation-specific parameters
    "nspin": 10,     # [yrs] number of years for spinup
    "bndry_flag": ".false.", # [".true." | ".false."] true for explicit boundary layer computation (presently only for sigma coord climate models)
    "sigma": 0.9925561, # sigma level for boundary flag
    "wb_flag": ".false.", # [".true." | ".false."] true for variable lake depth
    "iceflag": ".false.", # [".true." | ".false."] true for variable ice cover
    "s_flag": ".false.", # [".true." | ".false."] true for variable salinity
    "o18flag": ".false.", # [".true." | ".false."] true for variable d18O
    "deutflag": ".false.", # [".true." | ".false."] true for variable dD
    "z_screen": 5.0,  # [m] height of meteorological inputs
}
    

In [36]:
file_path = os.path.join(input_values['outdir'], input_values['lake_inc_template'])
# Read the file into a list of lines
with open(file_path, 'r') as file:
    lines = file.readlines()


In [37]:
# process the `parameter` values in each line
updated_lines = []
for line in lines:
    for key, value in input_values.items():
        if re.search(rf"\b{key}\b", line):
            if key in input_values['special_format_1']:
                tmp_value = f"'{value}'\n"
                line = re.sub(rf"(\b{key}\s*=\s*)([^,)]+)", rf"\g<1>{tmp_value}", line)
            else:
                # replace the numeric value for the matching key
                line = re.sub(rf"(\b{key}\s*=\s*)([^,)]+)", rf"\g<1>{value}", line)
                
    updated_lines.append(line)

# write updated lines to a new file
output_file = os.path.join(input_values['outdir'], input_values['casename'], input_values['lake_inc_filename'])
with open(output_file, "w") as file:
    file.writelines(updated_lines)

In [25]:
# ----------------